In [23]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as td
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import image
from matplotlib import pyplot
import time
import os
from google.colab import drive

import cv2
from torchmetrics import F1Score

In [25]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# path='/content/drive/My Drive/6721 Dataset/6721 Project dataset/'
path='/content/drive/My Drive/6721 Project dataset/'
path_save = '/content/drive/My Drive/6721 Dataset/'
print(os.listdir(path))

['Indian Traffic Signs Prediction(85 classes)', 'Persian Traffic Sign Dataset (PTSD)', 'Traffic Signs (GTSRB plus 162 custom classes)']


In [27]:
path_dataset1 = path+"Indian Traffic Signs Prediction(85 classes)/"
path_dataset2 = path+"Persian Traffic Sign Dataset (PTSD)/"
path_dataset3 = path+"Traffic Signs (GTSRB plus 162 custom classes)/Data_images/"

**Data Loaders**

---



In [28]:
def load_data(path_train, val_split, path_test, batch_size, input_size):
  
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    transform_train= transforms.Compose([transforms.Resize(input_size),
                                          #transforms.ColorJitter(brightness=(0.8,1.2), contrast=None, saturation=None, hue=None),
                                          transforms.ToTensor(),
                                          normalize
                                          ])

    transform_test = transforms.Compose([transforms.Resize(input_size),
                                         transforms.ToTensor(), 
                                         normalize])

    data_train = datasets.ImageFolder(root=path_train, transform=transform_train)
    data_test = datasets.ImageFolder(root=path_test, transform=transform_test)
    

    mappings = data_train.class_to_idx

    val_size = int(len(data_train)*val_split)
    train_size = len(data_train) - val_size

    train_dataset, val_dataset = td.random_split(data_train, [train_size, val_size])
    
    # class_weights = []
    # for root, subdir, files in os.walk(path_train):
    #     if len(files) > 0:
    #         class_weights.append(1/len(files))

    # sample_weights = [0] * len(data_train)

    # for idx, (data, label) in enumerate(data_train):
    #     class_weight = class_weights[label]
    #     sample_weights[idx] = class_weight

    # sampler = td.WeightedRandomSampler(sample_weights, num_samples=
    #                                 len(sample_weights), replacement=True)
    
    data_loader_train = td.DataLoader(train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      drop_last=False,
                                      num_workers=0,       
                                      pin_memory=True)
     
    data_loader_val = td.DataLoader(val_dataset,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    drop_last=False,
                                    num_workers=0) 
      
    data_loader_test = td.DataLoader(data_test,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=False,
                                   num_workers=0)
    
    return data_loader_train, data_loader_test, data_loader_val, mappings

Compute Class Weights

In [29]:
def get_image_size_dist(path):
    t1 = time.time()

    folders = os.listdir(path)
    total = 0

    class_samples = {}

    for folder in folders:
        samples = os.listdir(path+folder)

        class_samples[folder]=len(samples)

        print("Class: {} has samples: {} ".format(folder,len(samples)))
        total+=len(samples)

    print("\nTotal number of samples: ",total)

    print("\nTime taken: {}".format(time.time()-t1))

    return class_samples, total

In [30]:
def get_class_weights(path_train, mappings):

    class_samples, total = get_image_size_dist(path_train)

    class_weights = [0]*len(mappings.keys())

    for key in mappings.keys():
        class_weights[mappings[key]] = 1 - (class_samples[key]/ total)
      
    return class_weights

Training Setup

---



In [31]:
learning_rate = 0.001
num_epochs = 15

def get_criterion(class_weights):

    criterion = nn.CrossEntropyLoss(weight = class_weights)

    return criterion

In [32]:
def train_model(model, num_epochs, train_loader, criterion, optimizer, savepath, device):

      total_steps = len(train_loader)

      t1 = time.time()

      for epoch in range(num_epochs):
          for i, data in enumerate(train_loader):
              
              images, labels = data[0].to(device), data[1].to(device)
              
              model.train()

              # Forward pass
              outputs = model(images)
              loss = criterion(outputs, labels)
              
              # Backprop and optimisation
              optimizer.zero_grad()
              loss.backward()
              optimizer.step()
              
              # Train accuracy
              total = labels.size(0)
              _, predicted = torch.max(outputs.data, 1)
              correct = (predicted == labels).sum().item()
              
              if (i + 1) % 10 == 0:
                  model.eval() 
                  with torch.no_grad(): 
                      correctv = 0
                      totalv = 0
                      for datav in val_loader:
                          imagesv, labelsv = datav[0].to(device), datav[1].to(device)
                          outputsv = model(imagesv)
                          _, predictedv = torch.max(outputsv.data, 1)
                          totalv += labelsv.size(0)
                          correctv += (predictedv == labelsv).sum().item()
                          
                      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%, Validation Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                          (correct / total) * 100,
                          (correctv / totalv) * 100))
                      
                  
      print("######## Training Finished in {} seconds ###########".format(time.time()-t1))

      print("/n/n Saving model at: ",savepath)
      torch.save(model.state_dict(), savepath)

      return model, device

In [33]:
def evaluate_model(model, test_loader, device,nclasses):
   
    model.eval() 

    Y=[]
    y=[]

    with torch.no_grad(): 
        correct = 0
        total = 0
        for data in test_loader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            Y.extend(labels)
            y.extend(predicted)

        acc = (correct / total) * 100
        f1 = F1Score(num_classes=nclasses)
        f1score = f1(torch.IntTensor(y),torch.IntTensor(Y))

        print('Model Evaluation Results on {} test samples'.format(total))
        print('Test Accuracy: ', acc)
        print('Test F1 Score: ',f1score)
        

Architecture 1: AlexNet

On dataset 1:

In [34]:
nclasses=15

In [35]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', weights=None)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [36]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [37]:
AlexNet_model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [38]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [39]:
path_train = path_dataset1+"train/"
path_test = path_dataset1+"test/"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))
AlexNet_model.to(device)

Device: cuda:0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [47]:
class_weights = get_class_weights(path_train, mappings)

class_weights = torch.FloatTensor(class_weights)
class_weights = class_weights.to(device)

criterion = get_criterion(class_weights)

Class: CROSS_ROAD has samples: 140 
Class: GAP_IN_MEDIAN has samples: 180 
Class: COMPULSARY_KEEP_RIGHT has samples: 223 
Class: HORN_PROHIBITED has samples: 160 
Class: HUMP_OR_ROUGH_ROAD has samples: 101 
Class: LEFT_TURN_PROHIBITED has samples: 126 
Class: NO_ENTRY has samples: 174 
Class: NO_STOPPING_OR_STANDING has samples: 242 
Class: PEDESTRIAN_CROSSING has samples: 121 
Class: SPEED_LIMIT_40 has samples: 170 
Class: SPEED_LIMIT_80 has samples: 192 
Class: SPEED_LIMIT_30 has samples: 238 
Class: SPEED_LIMIT_50 has samples: 200 
Class: SPEED_LIMIT_60 has samples: 190 
Class: SPEED_LIMIT_70 has samples: 160 

Total number of samples:  2617

Time taken: 0.049219369888305664


In [50]:
savepath = path_save+"alexnetmodel_d1.pt"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, savepath, device)

KeyboardInterrupt: ignored

In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Test Accuracy of the model on the 710 test images: 19.859154929577468 %


On dataset 2:

In [ ]:
nclasses=12

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', weights=None)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train/"
path_test = path_dataset2+"test/"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

[0.007142857142857143, 0.005555555555555556, 0.004484304932735426, 0.00625, 0.009900990099009901, 0.007936507936507936, 0.005747126436781609, 0.004132231404958678, 0.008264462809917356, 0.0058823529411764705, 0.005208333333333333, 0.004201680672268907, 0.005, 0.005263157894736842, 0.00625]
[0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.00

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"alexnetmodel_d2.pt"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, savepath)

Device: cuda:0
Epoch [1/1], Step [10/66], Loss: 2.7568, Accuracy: 0.00%, Validation Accuracy: 10.33%
Epoch [1/1], Step [20/66], Loss: 2.6704, Accuracy: 0.00%, Validation Accuracy: 10.33%
Epoch [1/1], Step [30/66], Loss: 2.6873, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/1], Step [40/66], Loss: 2.6864, Accuracy: 15.62%, Validation Accuracy: 9.18%
Epoch [1/1], Step [50/66], Loss: 2.6760, Accuracy: 3.12%, Validation Accuracy: 9.18%
Epoch [1/1], Step [60/66], Loss: 2.5726, Accuracy: 15.62%, Validation Accuracy: 15.30%
######## Training Finished in 235.3470482826233 seconds ###########


In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Test Accuracy of the model on the 710 test images: 19.859154929577468 %


On dataset 3:

In [ ]:
nclasses=8

In [ ]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', weights=None)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
prev_out = AlexNet_model.classifier[4].out_features
AlexNet_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(AlexNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"

val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

[0.007142857142857143, 0.005555555555555556, 0.004484304932735426, 0.00625, 0.009900990099009901, 0.007936507936507936, 0.005747126436781609, 0.004132231404958678, 0.008264462809917356, 0.0058823529411764705, 0.005208333333333333, 0.004201680672268907, 0.005, 0.005263157894736842, 0.00625]
[0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.007142857142857143, 0.00

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"alexnetmodel_d3.pt"

trained_AlexNet_model, device = train_model(AlexNet_model, num_epochs, train_loader, criterion, optimizer, savepath)

Device: cuda:0
Epoch [1/1], Step [10/66], Loss: 2.7568, Accuracy: 0.00%, Validation Accuracy: 10.33%
Epoch [1/1], Step [20/66], Loss: 2.6704, Accuracy: 0.00%, Validation Accuracy: 10.33%
Epoch [1/1], Step [30/66], Loss: 2.6873, Accuracy: 9.38%, Validation Accuracy: 9.18%
Epoch [1/1], Step [40/66], Loss: 2.6864, Accuracy: 15.62%, Validation Accuracy: 9.18%
Epoch [1/1], Step [50/66], Loss: 2.6760, Accuracy: 3.12%, Validation Accuracy: 9.18%
Epoch [1/1], Step [60/66], Loss: 2.5726, Accuracy: 15.62%, Validation Accuracy: 15.30%
######## Training Finished in 235.3470482826233 seconds ###########


In [ ]:
# for loading model
# trained_AlexNet_model.load_state_dict(torch.load(path+"alexnetmodel.pt"))

In [ ]:
evaluate_model(trained_AlexNet_model, test_loader, device, nclasses)

Test Accuracy of the model on the 710 test images: 19.859154929577468 %


Architecture 2: VGG-11

on dataset 1:

In [ ]:
nclasses=15

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
VGG11_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train"
path_test = path_dataset1+"test"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"vgg11model_d1.pt"
trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

on dataset 2:

In [ ]:
nclasses=12

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train"
path_test = path_dataset2+"test"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"vgg11model_d2.pt"
trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

on dataset 3:

In [ ]:
nclasses=8

In [ ]:
VGG11_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
prev_out = VGG11_model.classifier[3].out_features
VGG11_model.classifier[6] = nn.Linear( prev_out, nclasses)

In [ ]:
optimizer = torch.optim.Adam(VGG11_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

KeyboardInterrupt: ignored

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"vgg11model_d3.pt"
trained_VGG11_model, device = train_model(VGG11_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_VGG11_model, test_loader, device, nclasses)

Architecture 3: ResNet-18

on dataset 1:

In [ ]:
nclasses=15

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
ResNet_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset1+"train"
path_test = path_dataset1+"test"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"resnetmodel_d1.pt"
trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

on dataset 2:

In [ ]:
nclasses=12

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset2+"train"
path_test = path_dataset2+"test"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"resnetmodel_d2.pt"
trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)

on dataset 3:

In [ ]:
nclasses=8

In [ ]:
ResNet_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', weights=None)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip


In [ ]:
ResNet_model.fc = nn.Linear( 512, nclasses)

In [ ]:
optimizer = torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate) 

In [ ]:
path_train = path_dataset3+"Train/"
path_test = path_dataset3+"Test/"
val_split = 0.2
batch_size = 128
input_size = (224,224)

train_loader, test_loader, val_loader, mappings = load_data(path_train, val_split, path_test, batch_size, input_size)

In [ ]:
class_weights = get_class_weights(path_train, mappings)
criterion = get_criterion(class_weights)

In [ ]:
savepath = path_save+"resnetmodel_d3.pt"
trained_ResNet_model, device = train_model(ResNet_model, num_epochs, train_loader, criterion, optimizer, savepath)

In [ ]:
evaluate_model(trained_ResNet_model, test_loader, device, nclasses)